In [1]:
import database
import importlib
import os
from datetime import datetime

from snowballing.operations import load_work, load_citations, reload
from snowballing.operations import work_by_varname, load_work_map_all_years
from snowballing.models import Place, DB
from snowballing.jupyter_utils import idisplay, work_button, new_button
from snowballing.selenium_scholar import SeleniumScholarQuerier
from snowballing.dbmanager import set_attribute

## Check Place

In [2]:
reload()
result = [
    idisplay(work_button(w))
    for _, w in load_work_map_all_years()
    if hasattr(w, "place1")
    or not isinstance(w.place, Place)
]
len(result)

0

In [3]:
[p for p in DB.places() if not hasattr(p, "type")]

[]

In [4]:
[(p, p.type) for p in DB.places()
 if p.type not in {
     "Conference", 
     "Journal", 
     "Magazine", 
     "Sponsor",
     "Proceedings",
     "Archive",
     "Tech Report",
     "Unpublished",
     "Thesis",
     "Patent",
     "Lang",
     "Web",
     "Standard",
     "Book",
     "Other",
 }]

[]

## Check Work

In [5]:
reload()
result = [
    idisplay(work_button(w))
    for _, w in load_work_map_all_years()
    if w.category == "work"
]
len(result)

0

## Check Due

In [6]:
reload()
result = [
    idisplay(work_button(w))
    for _, w in load_work_map_all_years()
    if w.category == "unrelated"
    and not hasattr(w, "due")
]
len(result)

HBox(children=(Button(description='martino2016a', style=ButtonStyle()),))

HBox(children=(Button(description='ferrucci2015a', style=ButtonStyle()),))

HBox(children=(Button(description='francese2015a', style=ButtonStyle()),))

HBox(children=(Button(description='minku2015a', style=ButtonStyle()),))

HBox(children=(Button(description='mittas2015a', style=ButtonStyle()),))

...

HBox(children=(Button(description='lokan2006a', style=ButtonStyle()),))

HBox(children=(Button(description='mair2005a', style=ButtonStyle()),))

HBox(children=(Button(description='mendes2005c', style=ButtonStyle()),))

HBox(children=(Button(description='premraj2005a', style=ButtonStyle()),))

53

## Check Name

In [56]:
reload()
result = [
    idisplay(work_button(w))
    for _, w in load_work_map_all_years()
    if not hasattr(w, "name")
    or w.name.endswith(".")
]
len(result)

0

## Check Authors

In [7]:
reload()
result = [
    idisplay(work_button(w))
    for _, w in load_work_map_all_years()
    if not hasattr(w, "authors")
    or w.authors.endswith(".")
    and not w.authors.endswith(" al.")
    and not w.authors[-3] in "- " and w.authors[-2].isalpha()
]
len(result)

HBox(children=(Button(description='conte1986a', style=ButtonStyle()),))

1

## Check File/Link

In [58]:
def check_file(arg):
    k, w = arg
    if hasattr(w, "link") or w.category in ("nofile", "site") or w.place.name == "Patent":
        return (k, w, "")
    if isinstance(getattr(w, "file"), str):
        if os.path.exists(os.path.join("files", w.file.split("#")[0])):
            return (k, w, "")
        return (k, w, "inexistent")
    if w.file is None:
        return (k, w, "no attribute")
    return (k, w, "type error")

reload()
result = [
    idisplay(work_button(w), r)
    for _, w, r in map(check_file, load_work_map_all_years())
    if r
]
len(result)

AttributeError: 'WorkOk' object has no attribute 'file'

In [26]:
declared_files = {
    w.file.split("#")[0] for k, w in load_work_map_all_years()
    if isinstance(w.file, str)
}
set(os.listdir('files')) - declared_files - {'alternatives'}

AttributeError: 'WorkOk' object has no attribute 'file'

## Check nofile

In [42]:
from IPython.display import display
reload()
result = [
    idisplay(work_button(w))
    for _, w in load_work_map_all_years()
    if w.category == "nofile"
    if getattr(w, "request", "") not in ("done", "wont", "researchgate")
    if w.place.name != "Patent"
]
len(result)

0

## Scholar

In [8]:
querier = None
reload()
worklist = sorted(
    [k for k, w in load_work_map_all_years() if not getattr(w, "scholar_ok", False)],
    key=lambda x: (int(x[-5:-1]), x)
)
len(worklist)

55

In [9]:
if worklist and querier is None:
    querier = SeleniumScholarQuerier()
    querier.apply_settings(10, 4)

[ INFO]  settings applied


In [10]:
from snowballing.snowballing import ScholarUpdate
supdate = ScholarUpdate(querier, worklist, force=False)

In [11]:
supdate

VBox(children=(HBox(children=(Button(description='Previous Work', disabled=True, icon='arrow-left', style=ButtonStyle()), Button(description='Reload', icon='refresh', style=ButtonStyle()), Button(description='Next Work', icon='arrow-right', style=ButtonStyle()), ToggleButton(value=False, description='Debug'), ToggleButton(value=False, description='TextArea'), Label(value='0'))), Output(outputs=({'output_type': 'stream', 'text': 'conte1986a False\n', 'name': 'stdout'},))))

In [ ]:
# Temp
set_attribute('premraj2005a', 'scholar_ok', True)
None

In [46]:
{w.scholar_ok for k, w in load_work_map_all_years() if getattr(w, "scholar_ok", False)}

{True}

## Forward

Finding scholar cluster: http://webapps.stackexchange.com/questions/45333/how-to-create-a-citation-alert-for-a-paper-without-citation-in-google-scholar

In [37]:
def set_snowball_date_button(key, date):
    def click(w):
        set_attribute(key, "snowball", "datetime({0.year}, {0.month}, {0.day})".format(date))
    return new_button("Set date", click)
    
date = datetime(2017, 3, 6)
reload()
result = [
    idisplay(
        work_button(w),
        set_snowball_date_button(k, date),
        '"{}", citation_file="{}"'.format(k, w.citation_file) if hasattr(w, 'scholar') else w.name,
        label=False
    )
    for k, w in load_work_map_all_years()
    if hasattr(w, 'snowball')
    and w.category == "snowball"
    and w.snowball != date
]
len(result)

0

## Tracking

In [38]:
def set_tracking_button(key):
    def click(w):
        set_attribute(key, "tracking", "alert")
    return new_button("Set track", click)

reload()
result = [
    idisplay(
        work_button(w),
        set_tracking_button(k),
        "https://scholar.google.com/scholar?cluster={}&hl=en&as_sdt=2005&sciodt=0,5".format(w.cluster_id) if hasattr(w, 'cluster_id') else w.name,
        getattr(w, 'scholar', ''),
        label=False,
    )
    for k, w in load_work_map_all_years()
    if getattr(w, "tracking", "") not in ("alert", "impossible")
    and w.category == "snowball"
]
len(result)

0

In [39]:
len({k for k, w in load_work_map_all_years() if w.category == "snowball"})

0

In [40]:
reload()
result = [
    idisplay(work_button(w))
    for k, w in load_work_map_all_years()
    if w.category == "snowball"
    if w.place.name in ("Tech Report", "Unpublished")
]
len(result)

0

## Dash

In [41]:
def check_dash(ele):
    k, w = ele
    def create_button(attr):
        def click(_):
            set_attribute(k, attr, str(getattr(w, attr)).replace("-", "--"))
        return new_button("Set " + attr, click)
    reasons = []
    if w.place.name in ("Thesis", "Tech Report"):
        return k, w, reasons
    if isinstance(getattr(w, 'pp', ''), int):
        reasons.append(create_button('pp'))
    elif getattr(w, 'pp', '').count('-') not in (0, 2):
        reasons.append(create_button('pp'))
    if isinstance(getattr(w, 'volume', ''), int):
        reasons.append(create_button('volume'))
    elif getattr(w, 'volume', '').count('-') not in (0, 2):
        reasons.append(create_button('volume'))
    if isinstance(getattr(w, 'number', ''), int):
        reasons.append(create_button('number'))
    elif getattr(w, 'number', '').count('-') not in (0, 2):
        reasons.append(create_button('number'))
    return k, w, reasons

reload()
result = [
    idisplay(work_button(w), *r)
    for k, w, r in map(check_dash, load_work_map_all_years())
    if r
]
len(result)

0